## import library

In [ ]:
import os
import numpy as np
import tensorflow as tf
from sklearn.neighbors import NearestNeighbors
from multiprocessing import Pool
from skimage.transform import resize
import random
import skimage as sk
import skimage.io
from multiprocessing import Pool
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import offsetbox
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from sklearn import manifold
import random
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications import VGG16
import matplotlib.pyplot as plt
from matplotlib import offsetbox
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from sklearn import manifold
from multiprocessing import Pool
from skimage.transform import resize
import skimage.io
from multiprocessing import Pool
import tensorflow as tf
from sklearn.neighbors import NearestNeighbors
import copy
import sklearn.metrics as metric
from statistics import mean
import re


In [ ]:
!pip install scikit-image 

## Data

In [ ]:
! git clone https://github.com/annisafitria/ImageRetrieval_219-231.git

Cloning into 'ImageRetrieval_219-231'...
remote: Enumerating objects: 1550, done.
remote: Counting objects: 100% (1550/1550), done.
remote: Compressing objects: 100% (1523/1523), done.
remote: Total 1550 (delta 66), reused 1443 (delta 16), pack-reused 0
Receiving objects: 100% (1550/1550), 102.60 MiB | 26.19 MiB/s, done.
Resolving deltas: 100% (66/66), done.
Checking out files: 100% (2045/2045), done.


## preprocessing

In [ ]:
class CV_IO_utils:
  def read_img(filePath):
      return skimage.io.imread(filePath, as_gray=True)

  def read_imgs_dir(dirPath, extensions, parallel=True):
      args = [os.path.join(dirPath, filename)
              for filename in os.listdir(dirPath)
              if any(filename.lower().endswith(ext) for ext in extensions)]
      if parallel:
          pool = Pool()
          imgs = pool.map(CV_IO_utils.read_img, args)
          pool.close()
          pool.join()
      else:
          imgs = [read_img(arg) for arg in args]
      return imgs

  # simpan gambar
  def save_img(filePath, img):
      skimage.io.imsave(filePath, img)

In [ ]:
class CV_transform_utils:
    
  def apply_transformer(imgs, transformer, parallel=True):
      if parallel:
          pool = Pool()
          imgs_transform = pool.map(transformer, [img for img in imgs])
          pool.close()
          pool.join()
      else:
          imgs_transform = [transformer(img) for img in imgs]
      return imgs_transform

  # Normalize 
  def normalize_img(img):
      return img / 255.

  # Resize image
  def resize_img(img, shape_resized):
      img_resized = resize(img, shape_resized,
                          anti_aliasing=True,
                          preserve_range=True)
      assert img_resized.shape == shape_resized
      return img_resized

  # Flatten image
  def flatten_img(img):
      return img.flatten("C")

##Feature Extraction

In [ ]:
correct = 0
wrong = 0
loss_ = []
acc_ = []
ground_truth = []
retrieval_labels = []
prec_list = []
rec_list = []

acc_transfer = []
loss_transfer = []
prec_transfer = []
rec_transfer = []


retrieval_label=os.listdir(r"/content/ImageRetrieval_219-231/dataset/train")
truth_label=os.listdir(r"/content/ImageRetrieval_219-231/dataset/test")

# Apply transformations to all images
class ImageTransformer(object):

    def __init__(self, shape_resize):
        self.shape_resize = shape_resize

    def __call__(self, img):
        img_transformed = CV_transform_utils.resize_img(img, self.shape_resize)
        img_transformed = CV_transform_utils.normalize_img(img_transformed)
        return img_transformed



# Run mode:  (transfer learning -> VGG19)
modelName = "VGG19" 
trainModel = True
parallel = True 

# Make paths
dataTrainDir = os.path.join(os.getcwd(), "/content/ImageRetrieval_219-231/dataset/train")
dataTestDir = os.path.join(os.getcwd(), "/content/ImageRetrieval_219-231/dataset/test")
outDir = os.path.join(os.getcwd(), "output", modelName)
if not os.path.exists(outDir):
    os.makedirs(outDir)

# Read images
extensions = [".png"]
print("Reading train images from '{}'...".format(dataTrainDir))
imgs_train = CV_IO_utils.read_imgs_dir(dataTrainDir, extensions, parallel=parallel)
print("Reading test images from '{}'...".format(dataTestDir))
imgs_test = CV_IO_utils.read_imgs_dir(dataTestDir, extensions, parallel=parallel)
shape_img = imgs_train[0].shape
print("Image shape = {}".format(shape_img))

Reading train images from '/content/ImageRetrieval_219-231/dataset/train'...
Reading test images from '/content/ImageRetrieval_219-231/dataset/test'...
Image shape = (495, 615)


In [ ]:
shape_img

(495, 615)

In [ ]:
print("Loading VGG19 pre-trained model...")
model = tf.keras.applications.VGG19(weights='imagenet', include_top=False,
                                            input_shape=(250, 250, 3))
model.summary()

Loading VGG19 pre-trained model...
80150528/80134624 [==============================] - 1s 0us/step
Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 250, 250, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 250, 250, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 250, 250, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 125, 125, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 125, 125, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 125, 125, 128)     147584    
                           

In [ ]:
shape_img_resize = tuple([int(x) for x in model.input.shape[1:]])
input_shape_model = tuple([int(x) for x in model.input.shape[1:]])
output_shape_model = tuple([int(x) for x in model.output.shape[1:]])
n_epochs = None


# Print some model info
print("input_shape_model = {}".format(input_shape_model))
print("output_shape_model = {}".format(output_shape_model))


transformer = ImageTransformer(shape_img_resize)
print("Applying image transformer to training images...")
imgs_train_transformed = CV_transform_utils.apply_transformer(imgs_train, transformer, parallel=parallel)
print("Applying image transformer to test images...")
imgs_test_transformed = CV_transform_utils.apply_transformer(imgs_test, transformer, parallel=parallel)

# Convert images to numpy array
X_train = np.array(imgs_train_transformed).reshape((-1,) + input_shape_model)
X_test = np.array(imgs_test_transformed).reshape((-1,) + input_shape_model)
print(" -> X_train.shape = {}".format(X_train.shape))
print(" -> X_test.shape = {}".format(X_test.shape))



# Create embeddings using model
print("Inferencing embeddings using pre-trained model...")
E_train = model.predict(X_train)
E_train_flatten = E_train.reshape((-1, np.prod(output_shape_model)))
E_test = model.predict(X_test)
E_test_flatten = E_test.reshape((-1, np.prod(output_shape_model)))
print(" -> E_train.shape = {}".format(E_train.shape))
print(" -> E_test.shape = {}".format(E_test.shape))
print(" -> E_train_flatten.shape = {}".format(E_train_flatten.shape))
print(" -> E_test_flatten.shape = {}".format(E_test_flatten.shape))


# Fit kNN model on training images
print("Fitting k-nearest-neighbour model on training images...")
knn = NearestNeighbors(n_neighbors=10, metric="cosine")
knn.fit(E_train_flatten)

# Perform image retrieval on test images
print("Performing image retrieval on test images...")
for i, emb_flatten in enumerate(E_test_flatten):
    _, indices = knn.kneighbors([emb_flatten]) # find k nearest train neighbours
    img_query = imgs_test[i] # query image
    imgs_retrieval = [imgs_train[idx] for idx in indices.flatten()] # retrieval images
    for x in indices.flatten():
      if truth_label[i][:3] in retrieval_label[x]:
        correct = correct + 1
      else:
        wrong = wrong + 1
    loss_.append((wrong/5))
    acc_.append((correct/5))
    correct = 0
    wrong = 0

    # Precision Recall
    for x in indices.flatten():
      ground_truth.append(re.split("(\d.*)", truth_label[i])[0])
      retrieval_labels.append(re.split("(\d.*)", retrieval_label[x])[0])
    
    rec_list.append(metric.recall_score(ground_truth, retrieval_labels, average='weighted'))
    prec_list.append(metric.precision_score(ground_truth, retrieval_labels, average='weighted'))

    ground_truth.clear()
    retrieval_labels.clear()


print("retrieval accuracy: {}".format(mean(acc_)))
print("retrieval loss: {}".format(mean(loss_)))
print("retrieval precision: {}".format(mean(prec_list)))
print("retrieval recall: {}".format(mean(rec_list)))

input_shape_model = (250, 250, 3)
output_shape_model = (7, 7, 512)
Applying image transformer to training images...
Applying image transformer to test images...
 -> X_train.shape = (838, 250, 250, 3)
 -> X_test.shape = (150, 250, 250, 3)
Inferencing embeddings using pre-trained model...
 -> E_train.shape = (838, 7, 7, 512)
 -> E_test.shape = (150, 7, 7, 512)
 -> E_train_flatten.shape = (838, 25088)
 -> E_test_flatten.shape = (150, 25088)
Fitting k-nearest-neighbour model on training images...
Performing image retrieval on test images...


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and 

retrieval accuracy: 0.48533333333333334
retrieval loss: 1.5146666666666666
retrieval precision: 0.7266666666666667
retrieval recall: 0.126


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
